# How to train and evaluate a model on the publicly available datasets

### This notebook will show how to
- instantiate dataloader for the demo data
- instantiate pytorch model
- instantiate a trainer function
- train the model
- evaluate the model on the test set

### Imports

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

### Instantiate DataLoader

In [2]:
filenames = ['./data/lurz2020/static20457-5-9-preproc0', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': False,
                 'batch_size': 128,
                 'scale':1,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

# Instantiate State of the Art Model (SOTA)

In [3]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_dilation': 1,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': False,
  'use_avg_reg':False,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

## Configure Trainer

In [4]:
trainer_fn = "sensorium.training.standard_trainer"

trainer_config = {'max_iter': 100,
                 'verbose': False,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

# Run model training

In [5]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

Epoch 67: 100%|██████████| 35/35 [00:05<00:00,  5.98it/s]


# Evaluete the model on test set

In [6]:
from sensorium.utility.scores import get_correlations, get_signal_correlations, get_fev

In [7]:
correlations = get_correlations(model, dataloaders["test"], device="cuda")
signal_correlations = get_signal_correlations(model, dataloaders["test"], device="cuda")
fevs = get_fev(model, dataloaders["test"], device="cuda")

In [8]:
metrics = {"Correlation (single trial)": correlations.mean(), 
           "Correlation (mean)": signal_correlations.mean(), 
           "FEVE": fevs.mean()}

In [9]:
print("Results for the model:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.3f}")

Results for the model:
Correlation (single trial): 0.305
Correlation (mean): 0.553
FEVE: 0.316
